# Data Extraction

In [1]:
%%capture
!pip install pandas
!pip install BeautifulSoup
!pip install requests
!pip install selenium

In [2]:
%%capture
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

In [207]:
#imports
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)
import math
#import for requests
import requests

#import beautiful soup
from bs4 import BeautifulSoup

#we will use selenium webdriver
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import os

In [2]:
from datetime import datetime,timedelta

currentSecond= datetime.now().second
currentMinute = datetime.now().minute
currentHour = datetime.now().hour

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year

In [3]:
"""
Function to get the data from source page using Beautiful soup

Input : page is the current page

Output: a dataframe
"""
def get_table(df_prev, driver):
    #strUrl = page.current_url
    #soup = BeautifulSoup(driver.page_source, "html5")
    df_result = pd.read_html(driver.page_source)
    df_result = df_result[2]
    df_result = pd.concat([df_prev, df_result])
    return df_result

In [4]:
"""
Function to get the table row and table date to get the xpath values for table row and table data

Input : df_date: a dataframe of calender from the website
        date: A date we are looking for

Output: table row number and table data where we are looking for that date
"""
def get_tr_td(df_date, date):
    tr = 0
    td = 0
    for i,day in enumerate(df_date.columns):
        if len(df_date[df_date[day] == float(date)].index) != 0:
            tr = df_date[df_date[day] == float(date)].index.tolist()[0] +1
            td = i+1
    
    return tr, td

In [20]:
"""
Function to select the year based on the user input

Input : a year in form of int from user

Output: out the number related to that year
"""

def get_year(year):
    
    dict_of_years = { '2003' : 1, '2004':2, '2005':3, '2006':4, '2007':5, '2008':6, '2009':7,
                      '2010' : 8, '2011':9, '2012':10,'2013':11, '2014': 12, '2015':13,
                      '2016': 14, '2017': 15, '2018': 16, '2019':17, '2020':18, '2021':19, '2022':20
                    }
    #selecting year 2003
    year_num = dict_of_years[year]
    return year_num
    


In [6]:
#get 30 days period
today = datetime.today()
yesterday = today - timedelta(days=1)
one_week_ago = today - timedelta(days=7)
thirty_days_ago = today - timedelta(days=30)


In [ ]:
#Create an empty dataframe
df = pd.DataFrame()

#try:

#create webdriver instance
driver = webdriver.Chrome()

#get the url with login page
driver.get("https://www.hk33.com/en/user/login-register")
sleep(1)

#sign in with username and password
driver.find_element("name","email_or_username").send_keys("bekkandbutter@gmail.com")
driver.find_element("name","password").send_keys("bekk1234")
#click on the login button
driver.find_element("xpath","/html/body/div[1]/main/div[4]/div[1]/form/div[6]/div/div[1]/div[2]").click()
sleep(2)

#click on image on top left corner to go to main page
link = driver.find_element("id","header_logo").click()

#click on hkGoal
sleep(2)
driver.find_element("xpath","/html/body/div[1]/main/div[2]/div[2]/div[2]/a").click()

#click on to navigate
sleep(2)
driver.find_element("xpath","/html/body/div[1]/main/div[1]/table/tbody/tr/td[3]").click()
sleep(1)
driver.find_element("xpath","/html/body/div[1]/main/div[1]/div[3]/div[3]/div[1]/a[3]").click()
sleep(1)


################Select the number of entires to show ############################
#click on drop and select 200
sleep(1)
driver.find_element("xpath","/html/body/div[1]/main/div[6]/select").click()
sleep(1)
driver.find_element("xpath","/html/body/div[1]/main/div[6]/select/option[3]").click()

#select a year to get the report from
year = input("Enter the year you want to retrieve the data for: ")
year_num = get_year(year) 


##############################Select timeline from calender###################################
driver.find_element("xpath","/html/body/div[1]/main/form/div[1]/div[1]/label/input").click()
sleep(1)
#select a year
driver.find_element("xpath",f"/html/body/div[4]/div/div/select[2]/option[{year_num}]").click()
sleep(1)
value = int(driver.find_element("xpath",f"/html/body/div[4]/div/div/select[1]/option[1]").get_attribute("value")) + 1    
print("value", driver.find_element("xpath",f"/html/body/div[4]/div/div/select[1]/option[1]").text)
                                    
for month_num in range(1,14-value):
        
        print("month ",month_num)
        
        #right calender - end month
        driver.find_element("xpath","/html/body/div[1]/main/form/div[1]/div[2]/label/input").click()
        sleep(1)
        #select the month
        driver.find_element("xpath",f"/html/body/div[4]/div/div/select[1]/option[{month_num}]").click()
        sleep(1)
        #select a year
        driver.find_element("xpath",f"/html/body/div[4]/div/div/select[2]/option[{year_num}]").click()
        sleep(1)
                            
       
        #date selection (first of each month)
        df_date = pd.read_html(driver.page_source)[3]  
        if value in [1,3,5,7,8,10,12]:
            tr, td = get_tr_td(df_date, 31)
            driver.find_element("xpath",f"/html/body/div[4]/table/tbody/tr[{tr}]/td[{td}]/a").click()
            print("31s")
            sleep(1)
        elif value in [4,6,9,11]:
            tr, td = get_tr_td(df_date, 30)
            driver.find_element("xpath",f"/html/body/div[4]/table/tbody/tr[{tr}]/td[{td}]/a").click()
            print("30s")
            sleep(1)
        elif value ==2 and year_num in [2,6,10,14,18]:
            tr, td = get_tr_td(df_date, 29)
            driver.find_element("xpath",f"/html/body/div[4]/table/tbody/tr[{tr}]/td[{td}]/a").click()
            print("29s")
            sleep(1)
        else:
            tr, td = get_tr_td(df_date, 28)
            driver.find_element("xpath",f"/html/body/div[4]/table/tbody/tr[{tr}]/td[{td}]/a").click()
            print("28s")
            sleep(1)
        
        #increament value
        value = value+1
        print("vakue inside is ", value)
        ##############################Select a year###################################
        driver.find_element("xpath","/html/body/div[1]/main/form/div[1]/div[1]/label/input").click()
        sleep(1)

        #month = int(thirty_days_ago.month)
        #left Calender -Start month
        driver.find_element("xpath",f"/html/body/div[4]/div/div/select[1]/option[{month_num}]").click()
        sleep(1)
        #year on left calender
        driver.find_element("xpath",f"/html/body/div[4]/div/div/select[2]/option[{year_num}]").click()
        sleep(1)
        
        #date selection (first of each month)
        df_date = pd.read_html(driver.page_source)[3]
        tr, td = get_tr_td(df_date, 1)
        driver.find_element("xpath",f"/html/body/div[4]/table/tbody/tr[{tr}]/td[{td}]/a").click()
        sleep(0.5)
        
        #-------------------------------- End Date -----------------------------------------------

        #clicking on Search Button
        
        driver.find_element("xpath","/html/body/div[1]/main/form/div[4]/div[1]").click()
        sleep(2)


        ################ Count and Iterate through all the available pages #######################
        
        #get the number of pages
        try:
            result = BeautifulSoup(driver.page_source, "html5")
            pages = result.find("div", class_="pagination_pages")
            count_pages =[page for i,page in enumerate(pages) if i%2 !=0]
            for i,num in enumerate(reversed(count_pages)):
                driver.find_element("xpath",f"/html/body/div[1]/main/div[9]/div[3]/div[{num.text}]").click()

                if len(count_pages)-i == len(count_pages) and month_num==1:
                    df_goal = get_table(df, driver)
                else:
                    df_goal = get_table(df_goal, driver)
        except:
            df_goal = get_table(df, driver)
            pass
        
    #except:
     #   pass
    
    
#except Exception:
#    print("Please try running the code again. Page crashed")
#    pass

In [446]:
df_goal.head()

球賽編號              開賽時間 賽事類別   主隊     客隊     百家平均初盤百家平均終盤馬會初盤馬會終盤  \
     球賽編號              開賽時間 賽事類別   主隊     客隊                        主   
0  TUE 21  2010-06-01 20:00  國際賽   澳洲     丹麥  2.9891 3.2394 ---  3.45   
1  TUE 23  2010-06-02 02:15  國際賽   瑞士  哥斯達黎加  1.5567 1.5682 ---  1.45   
2  TUE 22  2010-06-02 02:30  國際賽  葡萄牙    喀麥隆  1.6144 1.5866 ---  1.45   
3  TUE 24  2010-06-02 02:30  國際賽   荷蘭     加納  1.3446 1.3112 ---  1.26   
4  WED 21  2010-06-02 14:00   盃賽  濟州聯   蔚山現代  2.5250 2.5434 ---  2.50   

                                                     \
                         和                        客   
0  3.1975 3.2218 ---  3.25  2.2018 2.0940 ---  1.90   
1  3.6015 3.6272 ---  3.80  5.4885 5.4868 ---  5.85   
2  3.4906 3.5631 ---  3.65  5.1004 5.3839 ---  6.25   
3  4.3640 4.6131 ---  4.75  7.9155 8.6494 ---  8.50   
4  3.1566 3.1345 ---  3.25  2.5279 2.5529 ---  2.40   

        百家平均初盤 SD / CV百家平均終盤 SD / CV                                     ...  \
                                   主                                  和  ...   
0  0.2373 /  0.0794 0.2718 /  0.0839  0.1241 /  0.0388 0.1353 /  0.0420  ...   
1  0.0382 /  0.0245 0.0530 /  0.0338  0.1682 /  0.0467 0.1687 /  0.0465  ...   
2  0.0569 /  0.0352 0.0459 /  0.0289  0.1649 /  0.0472 0.1772 /  0.0497  ...   
3  0.0472 /  0.0351 0.0359 /  0.0274  0.3930 /  0.0901 0.4313 /  0.0935  ...   
4  0.0607 /  0.0240 0.0802 /  0.0315  0.0866 /  0.0274 0.0821 /  0.0262  ...   

  馬會大細初盤馬會大細終盤            馬會角球初盤馬會角球終盤         角球數     半場比數   全場比數 賽果HAD  賠率  
             大          細           球數   大   細 角球數     半場比數   全場比數 賽果HAD  賠率  
0    ---  1.95  ---  1.75          NaN NaN NaN NaN  (0 : 0)  1 : 0     H NaN  
1    ---  1.80  ---  1.90          NaN NaN NaN NaN  (0 : 0)  0 : 1     A NaN  
2    ---  2.05  ---  1.68          NaN NaN NaN NaN  (1 : 0)  3 : 1     H NaN  
3    ---  1.65  ---  2.10          NaN NaN NaN NaN  (1 : 0)  4 : 1     H NaN  
4    ---  2.00  ---  1.72          NaN NaN NaN NaN  (2 : 0)  3 : 1     H NaN  

[5 rows x 25 columns]

In [447]:
df_goal.shape

(4759, 25)

#### Export Raw data to excel

In [448]:
#export to excel the raw data

cwd = os.getcwd()
writer = pd.ExcelWriter(cwd + "/goal_data.xlsx", engine='openpyxl')
df_goal.to_excel(writer, sheet_name='Sheet1')
#df2.to_excel(writer, sheet_name='Sheet1', merge_cells = True, startrow=1)

writer.close()

#### Use the Saved excel file to load into Pandas Dataframe and preprocess the data

In [449]:
cwd = os.getcwd()
df_practice = pd.read_excel(cwd+"/goal_data.xlsx", index_col=0, header=[0,1])
df_practice.rename(columns={'球賽編號':'A球賽編號', '開賽時間':'B開賽時間','賽事類別':'C賽事類別','主隊':'D主隊','客隊':'E客隊',
                            '角球數':'AV角球數','賠率':'BB賠率','賽果HAD':'BA賽果HAD'
                           
                           }, inplace=True)
df_practice.head()

A球賽編號             B開賽時間 C賽事類別  D主隊    E客隊     百家平均初盤百家平均終盤馬會初盤馬會終盤  \
    A球賽編號             B開賽時間 C賽事類別  D主隊    E客隊                        主   
0  TUE 21  2010-06-01 20:00   國際賽   澳洲     丹麥  2.9891 3.2394 ---  3.45   
1  TUE 23  2010-06-02 02:15   國際賽   瑞士  哥斯達黎加  1.5567 1.5682 ---  1.45   
2  TUE 22  2010-06-02 02:30   國際賽  葡萄牙    喀麥隆  1.6144 1.5866 ---  1.45   
3  TUE 24  2010-06-02 02:30   國際賽   荷蘭     加納  1.3446 1.3112 ---  1.26   
4  WED 21  2010-06-02 14:00    盃賽  濟州聯   蔚山現代  2.5250 2.5434 ---  2.50   

                                                     \
                         和                        客   
0  3.1975 3.2218 ---  3.25  2.2018 2.0940 ---  1.90   
1  3.6015 3.6272 ---  3.80  5.4885 5.4868 ---  5.85   
2  3.4906 3.5631 ---  3.65  5.1004 5.3839 ---  6.25   
3  4.3640 4.6131 ---  4.75  7.9155 8.6494 ---  8.50   
4  3.1566 3.1345 ---  3.25  2.5279 2.5529 ---  2.40   

        百家平均初盤 SD / CV百家平均終盤 SD / CV                                     ...  \
                                   主                                  和  ...   
0  0.2373 /  0.0794 0.2718 /  0.0839  0.1241 /  0.0388 0.1353 /  0.0420  ...   
1  0.0382 /  0.0245 0.0530 /  0.0338  0.1682 /  0.0467 0.1687 /  0.0465  ...   
2  0.0569 /  0.0352 0.0459 /  0.0289  0.1649 /  0.0472 0.1772 /  0.0497  ...   
3  0.0472 /  0.0351 0.0359 /  0.0274  0.3930 /  0.0901 0.4313 /  0.0935  ...   
4  0.0607 /  0.0240 0.0802 /  0.0315  0.0866 /  0.0274 0.0821 /  0.0262  ...   

  馬會大細初盤馬會大細終盤            馬會角球初盤馬會角球終盤         AV角球數     半場比數   全場比數 BA賽果HAD  \
             大          細           球數   大   細 AV角球數     半場比數   全場比數 BA賽果HAD   
0    ---  1.95  ---  1.75          NaN NaN NaN   NaN  (0 : 0)  1 : 0       H   
1    ---  1.80  ---  1.90          NaN NaN NaN   NaN  (0 : 0)  0 : 1       A   
2    ---  2.05  ---  1.68          NaN NaN NaN   NaN  (1 : 0)  3 : 1       H   
3    ---  1.65  ---  2.10          NaN NaN NaN   NaN  (1 : 0)  4 : 1       H   
4    ---  2.00  ---  1.72          NaN NaN NaN   NaN  (2 : 0)  3 : 1       H   

  BB賠率  
  BB賠率  
0  NaN  
1  NaN  
2  NaN  
3  NaN  
4  NaN  

[5 rows x 25 columns]

In [450]:
#let's start splitting the data from the end columns first

df_practice[['AY全場比數','AZ全場比數']] = df_practice['全場比數']['全場比數'].str.split(':', expand=True)

In [451]:
%%capture
#dropping the column 全場比數
df_practice = df_practice.drop('全場比數', axis=1)
#df_practice.head()

In [452]:
%%capture
#splitting the column 半場比數
df_practice[['AW半場比數','AX半場比數']] = df_practice['半場比數']['半場比數'].str.split(':', expand=True)
df_practice['AW半場比數'] = df_practice['AW半場比數'].str.replace("(","")
df_practice['AX半場比數'] = df_practice['AX半場比數'].str.replace(")","")
df_practice = df_practice.drop('半場比數', axis=1)
df_practice.head()

#split the data by empty space
df_practice[['F主','I主','L主','O主']]= df_practice['百家平均初盤百家平均終盤馬會初盤馬會終盤']['主'].str.replace("  "," ").str.split(" ",expand=True)
df_practice[['G和','J和','M和','P和']]= df_practice['百家平均初盤百家平均終盤馬會初盤馬會終盤']['和'].str.replace("  "," ").str.split(" ",expand=True)
df_practice[['H客','K客','N客','Q客']]= df_practice['百家平均初盤百家平均終盤馬會初盤馬會終盤']['客'].str.replace("  "," ").str.split(" ",expand=True)
df_practice = df_practice.drop('百家平均初盤百家平均終盤馬會初盤馬會終盤', axis=1, level=0)



df_practice['主1'] = df_practice['百家平均初盤 SD / CV百家平均終盤 SD / CV']['主'].str.replace(" / ", "")
df_practice[['R主','S主','X主','Y主']] = df_practice['主1'].str.split(" ",expand=True)
df_practice['和2'] = df_practice['百家平均初盤 SD / CV百家平均終盤 SD / CV']['和'].str.replace(" / ", "")
df_practice[['T和','U和','Z和','AA和']] = df_practice['和2'].str.split(" ",expand=True)
df_practice['客3'] = df_practice['百家平均初盤 SD / CV百家平均終盤 SD / CV']['客'].str.replace(" / ", "")
df_practice[['V客','W客','AB客','AC客']] = df_practice['客3'].str.split(" ",expand=True)
df_practice.drop('百家平均初盤 SD / CV百家平均終盤 SD / CV', axis=1, level=0, inplace=True)
df_practice = df_practice.drop(['主1','和2','客3'],axis=1)


#dealing with 馬會讓球初盤馬會讓球終盤 column
try:
    if df_practice['馬會讓球初盤馬會讓球終盤']['球數'].isnull().value_counts().index.item() == True & df_practice['馬會讓球初盤馬會讓球終盤']['大'].isnull().value_counts().index.item() == True & df_practice['馬會讓球初盤馬會讓球終盤']['細'].isnull().value_counts().index.item() == True:
        df_practice[['AD球數','AG球數']] = np.nan
        df_practice[['AE主','AH主']] = np.nan
        df_practice[['AF客','AI客']] = np.nan
        df_practice.drop('馬會讓球初盤馬會讓球終盤', axis=1, level=0, inplace=True)
except:

    df_practice[['AD球數','AG球數']] = df_practice['馬會讓球初盤馬會讓球終盤']['球數'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice[['AE主','AH主']] = df_practice['馬會讓球初盤馬會讓球終盤']['主'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice[['AF客','AI客']] = df_practice['馬會讓球初盤馬會讓球終盤']['客'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice.drop('馬會讓球初盤馬會讓球終盤', axis=1, level=0, inplace=True)


#dealing with 馬會大細初盤馬會大細終盤 column
if df_practice['馬會大細初盤馬會大細終盤']['球數'].isnull().value_counts().index.item() == True & df_practice['馬會大細初盤馬會大細終盤']['大'].isnull().value_counts().index.item() == True & df_practice['馬會大細初盤馬會大細終盤']['細'].isnull().value_counts().index.item() == True:
    df_practice[['AJ球數','AM球數']] = np.nan
    df_practice[['AK大','AN大']] = np.nan
    df_practice[['AL細','AO細']] = np.nan
    df_practice.drop('馬會大細初盤馬會大細終盤', axis=1, level=0, inplace=True)
else:   
    df_practice[['AJ球數','AM球數']] = df_practice['馬會大細初盤馬會大細終盤']['球數'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice[['AK大','AN大']] = df_practice['馬會大細初盤馬會大細終盤']['大'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice[['AL細','AO細']] = df_practice['馬會大細初盤馬會大細終盤']['細'].str.replace("  ", " ").str.split(" ",expand=True)
    df_practice.drop('馬會大細初盤馬會大細終盤', axis=1, level=0, inplace=True)


#dealing with 馬會角球初盤馬會角球終盤 column
if df_practice['馬會角球初盤馬會角球終盤']['球數'].isnull().value_counts().index.item() == True & df_practice['馬會角球初盤馬會角球終盤']['大'].isnull().value_counts().index.item() == True & df_practice['馬會角球初盤馬會角球終盤']['細'].isnull().value_counts().index.item() == True:
    df_practice[['AP球數','AS球數']] = np.nan
    df_practice[['AQ大','AT大']] = np.nan
    df_practice[['AR細','AU細']] = np.nan
    df_practice.drop('馬會角球初盤馬會角球終盤', axis=1, level=0, inplace=True)
else:
    df_practice[['AP球數','AS球數','dummy1']] = df_practice['馬會角球初盤馬會角球終盤']['球數'].str.split(" ",expand=True)
    df_practice[['AQ大','AT大','dummy2']] = df_practice['馬會角球初盤馬會角球終盤']['大'].str.split(" ",expand=True)
    df_practice[['AR細','AU細','dummy3']] = df_practice['馬會角球初盤馬會角球終盤']['細'].str.split(" ",expand=True)
    df_practice.drop(['馬會角球初盤馬會角球終盤','dummy1','dummy2','dummy3'], axis=1, level=0, inplace=True)

In [453]:
df_practice.head()

,A球賽編號,B開賽時間,C賽事類別,D主隊,E客隊,AV角球數,BA賽果HAD,BB賠率,AY全場比數,AZ全場比數,...,AK大,AN大,AL細,AO細,AP球數,AS球數,AQ大,AT大,AR細,AU細
,A球賽編號,B開賽時間,C賽事類別,D主隊,E客隊,AV角球數,BA賽果HAD,BB賠率,,,...,,,,,,,,,,
0,TUE 21,2010-06-01 20:00,國際賽,澳洲,丹麥,NaN,H,NaN,1,0,...,---,1.95,---,1.75,NaN,NaN,NaN,NaN,NaN,NaN
1,TUE 23,2010-06-02 02:15,國際賽,瑞士,哥斯達黎加,NaN,A,NaN,0,1,...,---,1.80,---,1.90,NaN,NaN,NaN,NaN,NaN,NaN
2,TUE 22,2010-06-02 02:30,國際賽,葡萄牙,喀麥隆,NaN,H,NaN,3,1,...,---,2.05,---,1.68,NaN,NaN,NaN,NaN,NaN,NaN
3,TUE 24,2010-06-02 02:30,國際賽,荷蘭,加納,NaN,H,NaN,4,1,...,---,1.65,---,2.10,NaN,NaN,NaN,NaN,NaN,NaN
4,WED 21,2010-06-02 14:00,盃賽,濟州聯,蔚山現代,NaN,H,NaN,3,1,...,---,2.00,---,1.72,NaN,NaN,NaN,NaN,NaN,NaN


## Now we will rearrange columns alphabetically

In [454]:
#get the column names
cols=[]
for col_names in df_practice.columns.sort_values().tolist():
    cols.append(col_names[0])

In [455]:
for i in range(len(cols)):
    print(i, cols[i])

0 AA和
1 AB客
2 AC客
3 AD球數
4 AE主
5 AF客
6 AG球數
7 AH主
8 AI客
9 AJ球數
10 AK大
11 AL細
12 AM球數
13 AN大
14 AO細
15 AP球數
16 AQ大
17 AR細
18 AS球數
19 AT大
20 AU細
21 AV角球數
22 AW半場比數
23 AX半場比數
24 AY全場比數
25 AZ全場比數
26 A球賽編號
27 BA賽果HAD
28 BB賠率
29 B開賽時間
30 C賽事類別
31 D主隊
32 E客隊
33 F主
34 G和
35 H客
36 I主
37 J和
38 K客
39 L主
40 M和
41 N客
42 O主
43 P和
44 Q客
45 R主
46 S主
47 T和
48 U和
49 V客
50 W客
51 X主
52 Y主
53 Z和


In [456]:
#rearrange the columns so that we get data from A to BB
cols = cols[26:27]  + cols[-25:] + cols[:26] + cols[27:29]

In [457]:
#get the final dataframe and then export it to csv
df_final = df_practice[cols]

In [458]:
df_final.head()

,A球賽編號,B開賽時間,C賽事類別,D主隊,E客隊,F主,G和,H客,I主,J和,...,AS球數,AT大,AU細,AV角球數,AW半場比數,AX半場比數,AY全場比數,AZ全場比數,BA賽果HAD,BB賠率
,A球賽編號,B開賽時間,C賽事類別,D主隊,E客隊,,,,,,...,,,,AV角球數,,,,,BA賽果HAD,BB賠率
0,TUE 21,2010-06-01 20:00,國際賽,澳洲,丹麥,2.9891,3.1975,2.2018,3.2394,3.2218,...,NaN,NaN,NaN,NaN,0,0,1,0,H,NaN
1,TUE 23,2010-06-02 02:15,國際賽,瑞士,哥斯達黎加,1.5567,3.6015,5.4885,1.5682,3.6272,...,NaN,NaN,NaN,NaN,0,0,0,1,A,NaN
2,TUE 22,2010-06-02 02:30,國際賽,葡萄牙,喀麥隆,1.6144,3.4906,5.1004,1.5866,3.5631,...,NaN,NaN,NaN,NaN,1,0,3,1,H,NaN
3,TUE 24,2010-06-02 02:30,國際賽,荷蘭,加納,1.3446,4.3640,7.9155,1.3112,4.6131,...,NaN,NaN,NaN,NaN,1,0,4,1,H,NaN
4,WED 21,2010-06-02 14:00,盃賽,濟州聯,蔚山現代,2.5250,3.1566,2.5279,2.5434,3.1345,...,NaN,NaN,NaN,NaN,2,0,3,1,H,NaN


In [459]:
#export to excel the final processed data

cwd = os.getcwd()
writer = pd.ExcelWriter(cwd + "/goal_data.xlsx", engine='openpyxl')
df_final.to_excel(writer, sheet_name='Sheet1')
#df2.to_excel(writer, sheet_name='Sheet1', merge_cells = True, startrow=1)
writer.close()

In [460]:
#export to csv
cwd = os.getcwd()
df_final.to_csv(cwd + "/final_goal_data.csv",encoding="utf-8-sig")